In [0]:
#spark.read.json('/mnt/formula1jf/raw/2021-03-21/results.json').createOrReplaceTempView("results_cutover")

In [0]:
%sql
--SELECT raceId, COUNT(1)
  --FROM results_cutover
  --GROUP BY raceId
  --ORDER BY raceId DESC;

In [0]:
#spark.read.json('/mnt/formula1jf/raw/2021-03-28/results.json').createOrReplaceTempView("results_w1")

In [0]:
%sql
--SELECT raceId, COUNT(1)
  --FROM results_w1
  --GROUP BY raceId
  --ORDER BY raceId DESC;

In [0]:
#spark.read.json('/mnt/formula1jf/raw/2021-04-18/results.json').createOrReplaceTempView("results_w2")

In [0]:
%sql
--SELECT raceId, COUNT(1)
  --FROM results_w2
  --GROUP BY raceId
  --ORDER BY raceId DESC;

## Ingest results.json file

1. Read the JSON file using the spark dataframe reader

In [0]:
from pyspark.sql.functions import col, lit

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
file_date= dbutils.widgets.get("p_file_date")

In [0]:
result_schema = "resultId INT, raceId INT, driverId INT, constructorId INT, number INT, grid INT, position INT, positionText STRING, positionOrder INT, points FLOAT, laps INT, time STRING, milliseconds INT, fastestLap INT, rank INT, fastestLapTime STRING, fastestLapSpeed STRING, statusId INT"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
result_df = spark.read \
  .schema(result_schema) \
  .json(f'{raw_folder_path}/{file_date}/results.json')
#display(result_df)

2. Process Dataframe

In [0]:
result_final_df = result_df.withColumnRenamed("resultId", "result_id") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("DriverId", "driver_id") \
    .withColumnRenamed("constructorId", "constructor_id") \
    .withColumnRenamed("positionText", "position_text") \
    .withColumnRenamed("positionOrder", "position_order") \
    .withColumnRenamed("fastestLap", "fastest_lap") \
    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
    .drop(col('statusId')) \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(file_date))
result_final_df = add_ingestion_date(result_final_df)
#display(result_final_df)

3. Write data to datalake as parquet

### Method 1

In [0]:
#result_final_df.select("race_id").distinct().collect()

# for race_id_list in result_final_df.select("race_id").distinct().collect():
#     if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
#         spark.sql(f"ALTER TABLE f1_processed.results DROP IF EXISTS PARTITION (race_id = {race_id_list.race_id})")

In [0]:
#result_final_df.write.mode("append").partitionBy('race_id').parquet(f"{processed_folder_path}/results")
# database
#result_final_df.write.mode("overwrite").partitionBy('race_id').format("parquet").saveAsTable("f1_processed.results")

#result_final_df.write.mode("append").partitionBy('race_id').format("parquet").saveAsTable("f1_processed.results")

### Method 2
More efficient, no need to delete partitions manually

In [0]:
# spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic") #only replace partitions with new data received

In [0]:
# result_final_df = result_final_df.select("result_id", "driver_id", "constructor_id", "number", "grid", "position", "position_text",
#                     "position_order", "points", "laps", "time", "milliseconds", "fastest_lap", "rank", "fastest_lap_time",
#                     "fastest_lap_speed", "data_source", "file_date", "ingestion_timestamp", "race_id") #race_id as last column

In [0]:
# if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
#     result_final_df.write.mode("overwrite").insertInto("f1_processed.results")
# else:
#     result_final_df.write.mode("overwrite").partitionBy('race_id').format("parquet").saveAsTable("f1_processed.results")

In [0]:
# testing created function
save_table(result_final_df, "f1_processed", "results", "race_id")

In [0]:
# %fs
# ls mnt/formula1jf/processed/results

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
SELECT race_id, COUNT(1)
  FROM f1_processed.results
  GROUP BY race_id
  ORDER BY race_id DESC;

In [0]:
%sql
-- DROP TABLE f1_processed.results